End of Week 1 Exercise

This notebook demonstrates how to interact with large language models using both OpenAI and Ollama APIs. Based on the user's input and selected model, the notebook routes the message to the corresponding backend and returns the generated response. It's a simple yet flexible interface to explore and compare model behavior across different providers.

In [ ]:
# imports
import os
import requests
from dotenv import load_dotenv

from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# set up clients
openai = OpenAI()
ollama_url = "http://localhost:11434/api/chat"
ollama_headers = {"Content-Type": "application/json"}
# ollama = OpenAI(base_url="http://localhost:11434/v1" , api_key="ollama")


In [ ]:
# here is the question; type over this to ask something new

default_question = """
Please explain what this code does and why:
yield from {word for s in sentences for word in s.split()}
"""

In [ ]:
# Here is the System Prompt

system_prompt = """
You are an expert tutor who explains complex topics in simple, clear, and engaging ways. Tailor your teaching style to the 
learner’s level of knowledge and preferred learning pace. Encourage critical thinking, provide examples, 
and ask occasional questions to check understanding. Avoid giving direct answers when guiding 
problem-solving — instead, offer hints or break the problem into steps. Be friendly, patient, and always supportive. 
Adapt your explanations based on feedback or confusion. When asked for code, equations, or definitions, 
provide them in a structured, easy-to-understand format.
"""

In [ ]:
# Here is the System Prompt
user_prompt = "Please give a detailed explanation to the following question: " 

In [ ]:
# Get User prompt

def get_user_prompt(question=None):
    if not question:
        print(f"As the given question is empty. So I'm going to ask a default question, which is {default_question}" )
        question = default_question
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt + question}
    ]
    return messages

In [ ]:
# Get Llama 3.2 to answer
def get_answer_from_ollama(question=None):
    messages = get_user_prompt(question)

    data = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": False
    }

    response = requests.post(ollama_url, headers=ollama_headers, json=data)
    display(Markdown(response.json()['message']['content']))

In [ ]:
# Get gpt-4o-mini to answer, with streaming

def get_answer_from_openai(question=None):
    messages = get_user_prompt(question)
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Ask a question and get the user's response
answer = input("What's the question you want to ask?")

model_choice = input("Please choose a model to use (GPT or Llama): ")

if model_choice.lower() == "gpt":
    get_answer_from_openai(answer)
elif model_choice.lower() == "llama":
    get_answer_from_ollama(answer)
else:
    print("Choose the correct model name")